In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.random_projection import GaussianRandomProjection
from sklearn.preprocessing import RobustScaler, StandardScaler, QuantileTransformer, FunctionTransformer

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression, Lasso, Ridge, SGDRegressor
from sklearn.linear_model import LogisticRegression
from scipy.stats import boxcox
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import (StandardScaler, MinMaxScaler,
                                   PolynomialFeatures)
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_predict, train_test_split, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [3]:
train_data = pd.read_csv('train.csv', sep = ';')
train_data.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,6.6,0.3,0.36,1.2,0.035,43,126,0.9909,3.01,0.63,11.4,6,white
1,7.7,0.5,0.26,1.9,0.062,9,31,0.9966,3.39,0.64,9.6,5,red
2,8.4,0.5,0.35,2.9,0.076,21,127,0.9976,3.23,0.63,9.2,5,red
3,7.5,0.4,0.33,5.0,0.045,30,131,0.9942,3.32,0.44,10.9,6,white
4,6.4,0.2,0.25,20.2,0.083,35,157,0.9998,3.17,0.50,9.1,5,white


In [4]:
def feat_eng(df):
    df.columns = df.columns.str.replace(' ', '_')
    df['total_acidity'] = df['fixed_acidity'] + df['volatile_acidity'] + df['citric_acid']
    df['acidity_to_pH_ratio'] = df['total_acidity'] / df['pH']
    df['free_sulfur_dioxide_to_total_sulfur_dioxide_ratio'] = df['free_sulfur_dioxide'] / df['total_sulfur_dioxide']
    df['alcohol_to_acidity_ratio'] = df['alcohol'] / df['total_acidity']
    df['residual_sugar_to_citric_acid_ratio'] = df['residual_sugar'] / df['citric_acid']
    df['alcohol_to_density_ratio'] = df['alcohol'] / df['density']
    df['total_alkalinity'] = df['pH'] + df['alcohol']
    df['total_minerals'] = df['chlorides'] + df['sulphates'] + df['residual_sugar']
    return df

In [5]:
train_data = feat_eng(train_data)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6714 entries, 0 to 6713
Data columns (total 21 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   fixed_acidity                                      6714 non-null   float64
 1   volatile_acidity                                   6714 non-null   float64
 2   citric_acid                                        6714 non-null   float64
 3   residual_sugar                                     6714 non-null   float64
 4   chlorides                                          6714 non-null   float64
 5   free_sulfur_dioxide                                6714 non-null   int64  
 6   total_sulfur_dioxide                               6714 non-null   int64  
 7   density                                            6714 non-null   float64
 8   pH                                                 6714 non-null   float64
 9   sulphate

In [6]:
train_data_red = train_data[train_data['type'] == 'red']
train_data_red.head(5)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,...,quality,type,total_acidity,acidity_to_pH_ratio,free_sulfur_dioxide_to_total_sulfur_dioxide_ratio,alcohol_to_acidity_ratio,residual_sugar_to_citric_acid_ratio,alcohol_to_density_ratio,total_alkalinity,total_minerals
1,7.7,0.5,0.26,1.9,0.062,9,31,0.9966,3.39,0.64,...,5,red,8.46,2.495575,0.290323,1.134752,7.307692,9.632751,12.99,2.602
2,8.4,0.5,0.35,2.9,0.076,21,127,0.9976,3.23,0.63,...,5,red,9.25,2.863777,0.165354,0.994595,8.285714,9.222133,12.43,3.606
18,8.8,0.4,0.49,2.8,0.083,18,111,0.9982,3.30,0.60,...,5,red,9.69,2.936364,0.162162,0.980392,5.714286,9.517131,12.80,3.483
20,7.0,0.4,0.19,2.3,0.071,18,36,0.9948,3.39,0.56,...,5,red,7.59,2.238938,0.500000,1.436100,12.105263,10.956976,14.29,2.931
21,8.2,0.4,0.38,2.6,0.093,23,60,0.9979,3.31,0.71,...,6,red,8.98,2.712991,0.383333,1.213808,6.842105,10.922938,14.21,3.403


In [7]:
train_data_red = train_data_red.drop(columns = ['type'])

In [8]:
from scipy import stats
z = np.abs(stats.zscore(train_data_red))


C:\Users\IT tools\anaconda3\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [9]:
red_qualified = train_data_red[(z < 3).all(axis=1)]
red_qualified

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,total_acidity,acidity_to_pH_ratio,free_sulfur_dioxide_to_total_sulfur_dioxide_ratio,alcohol_to_acidity_ratio,residual_sugar_to_citric_acid_ratio,alcohol_to_density_ratio,total_alkalinity,total_minerals


In [10]:
X1 = train_data_red.drop(columns = 'quality')
Y1 = train_data_red['quality']


In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [8, 10, 12, 14],
    'max_features': [2, 3, 12],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
grid_search.fit(X1_s, Y1)

In [ ]:
train_data_white = train_data[train_data['type'] == 'white']
train_data_white.head()

In [ ]:
train_data_white = train_data_white.drop(columns = ['type'])

In [ ]:
y = np.abs(stats.zscore(train_data_white))

In [ ]:
white_qualified = train_data_white[(y < 3).all(axis=1)]
white_qualified

In [ ]:
X2 = white_qualified.drop(columns = 'quality')
Y2 = white_qualified['quality']

In [ ]:
s = StandardScaler()
X2_s = s.fit_transform(X2)

In [ ]:
pipeline_white = Pipeline(steps = [('scaler', StandardScaler()), ('pca', PCA(n_components=6)), ('polynomial', PolynomialFeatures(degree = 2)), ('random_forest', RandomForestRegressor(n_estimators = 300, 
                            criterion = 'squared_error', max_depth = 14, min_samples_leaf = 3, min_samples_split = 8, max_features = 12, bootstrap = True))])
               

In [ ]:
pipeline_white.fit(X2, Y2)

In [ ]:
test_data = pd.read_csv('test.csv', sep = ';')
test_data.head(5)

In [ ]:
test_red = test_data[test_data['type'] == 'red']
test_red.head(5)

In [ ]:
id1 = test_red['id']
test_red_qualified = test_red.drop(columns = ['id','type'])


In [ ]:
predicted1 = pipeline.predict(test_red_qualified)

In [ ]:
data = {'id': id1, 'quality': predicted1}
red_table = pd.DataFrame(data)
red_table.head(5)

In [ ]:
test_white = test_data[test_data['type'] == 'white']

In [ ]:
id2 = test_white['id']
test_white_qualified = test_white.drop(columns = ['id','type'])


In [ ]:
predicted2 = pipeline_white.predict(test_white_qualified)

In [ ]:
data2 = {'id': id2, 'quality': predicted2}
white_table = pd.DataFrame(data2)
white_table.head(5)

In [ ]:
table = pd.concat([red_table, white_table])
table

In [ ]:
csv_filename = 'results2.csv'
table.to_csv(csv_filename, index = False)
print(f'DataFrame saved to {csv_filename}')